In [17]:
import json
import pandas as pd
import requests

In [18]:
follows = pd.read_csv('../shared-folder-gald/data/follow-link.csv')

In [19]:
usernames = pd.unique(follows[['source', 'target']].values.ravel()).tolist()

In [32]:
with open("../shared-folder-gald/keys3.txt") as f:
    lines = f.readlines()
    client_key = lines[0].strip()
    client_secret = lines[1].strip()

In [33]:
def get_access_token(client_key, client_secret):
    # Endpoint URL
    endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"

    # Request headers
    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
    }

    # Request body parameters
    data = {
        'client_key': client_key,
        'client_secret': client_secret,
        'grant_type': 'client_credentials',
    }

    # Make the POST request
    response = requests.post(endpoint_url, headers=headers, data=data)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse and print the response JSON
        response_json = response.json()
        return response_json['access_token']
    else:
        # If the request was not successful, print the error response JSON
        
        return response.json()

In [34]:
# def get_access_token(client_key, client_secret):
#     endpoint_url = "https://open.tiktokapis.com/v2/oauth/token/"
#     headers = {'Content-Type': 'application/x-www-form-urlencoded'}
#     data = {
#         'client_key': client_key,
#         'client_secret': client_secret,
#         'grant_type': 'client_credentials',
#     }

#     response = requests.post(endpoint_url, headers=headers, data=data)
#     response_json = response.json()

#     if response.status_code == 200:
#         print(response_json)  # Debugging print
#         return response_json.get('access_token', 'No access token found')
#     else:
#         print(f"Error {response.status_code}: {response_json}")  # Debugging print
#         return response_json

In [35]:
def get_repost_info(username, access_token,fields ="id, username, favorites_count" ):
    query_params = {"fields": fields}
    query_body = {"username": username}
    headers = {"Authorization": f"Bearer {access_token}"}
    
    endpoint = "https://open.tiktokapis.com/v2/research/user/reposted_videos/"
    response = requests.post(endpoint, json=query_body, params=query_params, headers=headers)

    if response.status_code == 200:
        # Parse and extract information from the response
        return response.json().get("data", {})
    else:
        
        return {response.text}

In [36]:
access_token = get_access_token(client_key, client_secret)

KeyError: 'access_token'

keeping track with calls
27.02 - we ran keys2 unril 1000 

1.03 - keys2 1001:2002

1.03 - keys1 2002:3003



In [25]:
reposted_videos = []
for username in usernames[3003:4004]:
    reposted_videos.append(get_repost_info(username, access_token))

In [26]:
reposted_videos

[{'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 3096,
    'id': 7466954700168973614,
    'username': 'spencerpratt'},
   {'favorites_count': 9,
    'id': 7465893182291578143,
    'username': 'trueandclean'},
   {'id': 7465803147383000363,
    'username': 'chasebmarlin',
    'favorites_count': 17492},
   {'favorites_count': 63595,
    'id': 7459414815673486635,
    'username': 'kyratube_'},
   {'favorites_count': 17,
    'id': 7453141999592508715,
    'username': 'rayzorjohnson2'},
   {'id': 7472500490622766378,
    'username': 'maggswegner',
    'favorites_count': 5689},
   {'favorites_count': 4,
    'id': 7449785053946629407,
    'username': 'trueandclean'},
   {'id': 7464344741883186463,
    'username': 'm_green75',
    'favorites_count': 2524},
   {'favorites_count': 25971,
    'id': 7457646291225562398,
    'username': 'abcnews'},
   {'favorites_count': 3553,
    'id': 7467570329326456097,
    'username': 'ellishablake'},
   {'favorites_count': 81989,


In [27]:
#saving global reposts for weights in the network
import csv
with open('reposted_videos_raw', 'w') as f:
     
    # using csv.writer method from CSV package
    write = csv.writer(f)
    write.writerow(reposted_videos)

In [28]:
combined = {}
for i, username in enumerate(usernames[3003:4004]):
    combined[username] = reposted_videos[i]
    
combined 


{'bracilanewellness': {'cursor': 21,
  'has_more': True,
  'reposted_videos': [{'favorites_count': 3096,
    'id': 7466954700168973614,
    'username': 'spencerpratt'},
   {'favorites_count': 9,
    'id': 7465893182291578143,
    'username': 'trueandclean'},
   {'id': 7465803147383000363,
    'username': 'chasebmarlin',
    'favorites_count': 17492},
   {'favorites_count': 63595,
    'id': 7459414815673486635,
    'username': 'kyratube_'},
   {'favorites_count': 17,
    'id': 7453141999592508715,
    'username': 'rayzorjohnson2'},
   {'id': 7472500490622766378,
    'username': 'maggswegner',
    'favorites_count': 5689},
   {'favorites_count': 4,
    'id': 7449785053946629407,
    'username': 'trueandclean'},
   {'id': 7464344741883186463,
    'username': 'm_green75',
    'favorites_count': 2524},
   {'favorites_count': 25971,
    'id': 7457646291225562398,
    'username': 'abcnews'},
   {'favorites_count': 3553,
    'id': 7467570329326456097,
    'username': 'ellishablake'},
   {'favo

In [29]:
rows = []

for key, value in combined.items():
    if value and 'reposted_videos' in value:  # Ensure key has data
        for video in value['reposted_videos']:
            rows.append({
                'username': key,  # Dictionary key
                'video_id': video['id'],  # Video ID
                'creator': video['username'], # Video username
                'favourites': video["favorites_count"]
            })

# Create DataFrame
reposted_df = pd.DataFrame(rows)

# Display DataFrame
print(reposted_df)

               username             video_id           creator  favourites
0     bracilanewellness  7466954700168973614      spencerpratt        3096
1     bracilanewellness  7465893182291578143      trueandclean           9
2     bracilanewellness  7465803147383000363      chasebmarlin       17492
3     bracilanewellness  7459414815673486635         kyratube_       63595
4     bracilanewellness  7453141999592508715    rayzorjohnson2          17
...                 ...                  ...               ...         ...
9219           totalbau  7376252455497108742          jahte.rs           1
9220           totalbau  7260986491088719147  cheflucacorleone      349986
9221           totalbau  7258062204358954283        kelly_bove      174609
9222           totalbau  7238762864033402138    tips_tiger9304         590
9223          vertemode  7301461994757033262      tokkigifting           1

[9224 rows x 4 columns]


In [30]:
reposted_df.to_csv("reposted_videos.csv", mode="a", index=False, header=False)